In [213]:
import requests
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [40]:
def get_ids(search_term: str, limit: str = '50'):
    '''
    Função responsável por por listar os ids da pesquisa
    '''

    # Monta a URL
    url = f"https://api.mercadolibre.com/sites/MLA/search?q={search_term}&limit={limit}#json"
    # Usa a biblioteca requests para fazer a requisição e transformar os resultados em um objeto json
    response_json = requests.get(url).json()
    # Itera sobre o json para armazenar os ids em uma lista
    ids = list()
    for response in response_json['results']:
        ids.append(response['id'])

    return ids

In [201]:
def extrai_array(atributos, id):
    '''
    Função que extrai atributos dos campos de lista
    '''
    for atributo in atributos:
        if atributo.get('id') == id:
            return atributo.get('value_name')

In [244]:
def get_info(id):
    '''
    Função responsável por consultar resultados do produto pelo id
    e extrair informações relevantes do json
    '''

    # Consulta API para retornar resultados de acordo com o ID
    url = f'https://api.mercadolibre.com/items/{id}'
    print('Consultando informações da URL:', url)
    results = requests.get(url).json()

    # Filtrando as colunas relevantes
    filtered_results = {
        'id': results.get('id'),
        'site_id': results.get('site_id'),
        'title': results.get('title'),
        'seller_id': results.get('seller_id'),
        'category_id': results.get('category_id'),
        'price': results.get('price'),
        'original_price': results.get('original_price'),
        'currency_id': results.get('currency_id'),
        'initial_quantity': results.get('initial_quantity'),
        'accepts_mercadopago': results.get('accepts_mercadopago'),
        'status': results.get('status'),
        'shipping_local_pick_up': results.get('shipping', {}).get('local_pick_up'),
        'shipping_free_shipping': results.get('shipping', {}).get('free_shipping'),
        'seller_address_state': results.get('seller_address', {}).get('state', {}).get('name'),
        'seller_address_country': results.get('seller_address', {}).get('country', {}).get('name'),
        'warranty_time': extrai_array(results.get('sale_terms', []), 'WARRANTY_TIME'),
        'warranty_type': extrai_array(results.get('sale_terms', []), 'WARRANTY_TYPE'),
        'brand': extrai_array(results.get('attributes', []), 'BRAND'),
        'color': extrai_array(results.get('attributes', []), 'COLOR'),
        'depth': extrai_array(results.get('attributes', []), 'DEPTH'),
        'diameter': extrai_array(results.get('attributes', []), 'DIAMETER'),
        'functions': extrai_array(results.get('attributes', []), 'FUNCTIONS'),
        'height': extrai_array(results.get('attributes', []), 'HEIGHT'),
        'is_moisture_resistant': extrai_array(results.get('attributes', []), 'IS_MOISTURE_RESISTANT'),
        'item_condition': extrai_array(results.get('attributes', []), 'ITEM_CONDITION'),
        'manufacturer': extrai_array(results.get('attributes', []), 'MANUFACTURER'),
        'model': extrai_array(results.get('attributes', []), 'MODEL'),
        'ram_memory': extrai_array(results.get('attributes', []), 'RAM_MEMORY'),
        'operative_system': extrai_array(results.get('attributes', []), 'OPERATING_SYSTEM'),
        'operative_systems_compatibility': extrai_array(results.get('attributes', []), 'OPERATIVE_SYSTEMS_COMPATIBILITY'),
        'with_integrated_voice_assistants': extrai_array(results.get('attributes', []), 'WITH_INTEGRATED_VOICE_ASSISTANTS'),
        'virtual_assistants': extrai_array(results.get('attributes', []), 'VIRTUAL_ASSISTANTS'),
        'storage_capacity': extrai_array(results.get('attributes', []), 'STORAGE_CAPACITY'),
        'voltage': extrai_array(results.get('attributes', []), 'VOLTAGE'),
        'weight': extrai_array(results.get('attributes', []), 'WEIGHT'),
        'width': extrai_array(results.get('attributes', []), 'WIDTH'),
        'with_360_sound': extrai_array(results.get('attributes', []), 'WITH_360_SOUND'),
        'with_bluetooth': extrai_array(results.get('attributes', []), 'WITH_BLUETOOTH'),
        'with_camera': extrai_array(results.get('attributes', []), 'WITH_CAMERA'),
        'with_digital_display': extrai_array(results.get('attributes', []), 'WITH_DIGITAL_DISPLAY'),
        'with_recyclable_materials': extrai_array(results.get('attributes', []), 'WITH_RECYCLABLE_MATERIALS'),
        'with_wifi_connection': extrai_array(results.get('attributes', []), 'WITH_WI_FI_CONNECTION')
    }

    return filtered_results


In [210]:
ids = get_ids('chromecast')

In [ ]:
# Usando threads, é possível melhorar a performance com paralelismo
# Diminuição notada nos testes (50 itens): 20s para 5s 
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(get_info, ids)

In [264]:
url = 'https://api.mercadolibre.com/items/MLA1434031785'
response_json = requests.get(url).json()

In [265]:
response_json

{'id': 'MLA1434031785',
 'site_id': 'MLA',
 'title': 'Chromecast Google Ga03131 Tv Hd 8gb 2gb Ram 60 Fps Wifi Bluetooth Hdmi 4ta Gen Blanco',
 'seller_id': 621229920,
 'category_id': 'MLA352001',
 'official_store_id': 74930,
 'price': 112414.0,
 'base_price': 112414.0,
 'original_price': None,
 'currency_id': 'ARS',
 'initial_quantity': 280,
 'sale_terms': [{'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantía',
   'value_id': '2230280',
   'value_name': 'Garantía del vendedor',
   'value_struct': None,
   'values': [{'id': '2230280',
     'name': 'Garantía del vendedor',
     'struct': None}],
   'value_type': 'list'},
  {'id': 'WARRANTY_TIME',
   'name': 'Tiempo de garantía',
   'value_id': None,
   'value_name': '3 meses',
   'value_struct': {'number': 3, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '3 meses',
     'struct': {'number': 3, 'unit': 'meses'}}],
   'value_type': 'number_unit'},
  {'id': 'INVOICE',
   'name': 'Facturación',
   'value_id': '6891885',
   'value

In [256]:
df = pd.DataFrame(results)

df.head()

,id,site_id,title,seller_id,category_id,price,original_price,currency_id,initial_quantity,accepts_mercadopago,...,virtual_assistants,voltage,weight,width,with_360_sound,with_bluetooth,with_camera,with_digital_display,with_recyclable_materials,with_wifi_connection
0,MLA1434031785,MLA,Chromecast Google Ga03131 Tv Hd 8gb 2gb Ram 60...,621229920,MLA352001,112414.0,None,ARS,280,True,...,None,None,0.063 kg,61 mm,None,Sí,None,None,None,None
1,MLA1418251151,MLA,Google Chromecast 4 Generación Con Google Tv H...,250121682,MLA352001,134900.0,None,ARS,361,True,...,None,None,112 g,61 mm,None,Sí,None,None,None,None
2,MLA1520563158,MLA,Chromecast With Google Tv De Voz 4ª Generación...,1443311053,MLA352001,161900.0,None,ARS,264,True,...,None,None,0.3 g,12 cm,None,Sí,None,None,None,None
3,MLA1876051918,MLA,Chromecast Google Ga03131 Tv Hd 8gb 2gb Ram 60...,680515,MLA352001,144900.0,None,ARS,126,True,...,None,None,1 kg,None,None,None,None,None,None,None
4,MLA1108749509,MLA,Chromecast With Google Tv De Voz 4 Generación ...,207527715,MLA352001,161900.0,None,ARS,38,True,...,None,None,None,"2.4 """,None,Sí,None,None,None,None
